# RAG - LLM - Parsing - Chunking

Parsing is the process of extracting raw text from documents such as PDFs, .docx files, youtube videos and so on. It depends on the type of data you want to parse.

For this LLM, only pdfs will be parsed

## Load libraries

In [45]:
import pymupdf
import pymupdf4llm
import sys
from datetime import datetime
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

sys.path.append("..")

from gcp_utils.gcs import get_file
from rag_llm_energy_expert.config import GCP_CONFIG, LLM_CONFIG

In [46]:


model = SentenceTransformer("all-MiniLM-L6-v2", trust_remote_code=True)
# In case you want to reduce the maximum length:
#model.max_seq_length = 8192

queries = [
    "how much protein should a female eat",
    "summit define",
]
documents = [
    "As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
    "Definition of summit for English Language Learners. : 1  the highest point of a mountain : the top of a mountain. : 2  the highest level. : 3  a meeting or series of meetings between the leaders of two or more governments.",
]

#query_embeddings = model.encode(queries, prompt_name="query")
document_embeddings = model.encode(documents)

#scores = (query_embeddings @ document_embeddings.T) * 100
print(document_embeddings.shape)

(2, 384)


## Initialize config classes

In [47]:
gcp_config = GCP_CONFIG()
llm_config = LLM_CONFIG()

## Parsing PDFs

There are tons of libraries to extract data from PDFs, nevertheless, [*PyMuPDF*](https://pypi.org/project/pymupdf4llm/) is one of the best libraries because:

- Detects standard text and tables
- Header lines are identified via de font size and appropiately prefixed with one or more '#' tags.
- Bold, italic, mono-spaced text and code blocks are detected and formatted accordingly.
- By default, all document pages are processed.
- Support for pages with multiple text columns.
- Support image or vector graphic on the page and they're stored as an image.
- ***Support for page chunks***. Instead of returning one large string for the whole document, a list of dictionaries can be generated. One for each page.

*All the data parsed here comes from GCP*

Reading into memory is faster than download the pdf into a file system, and then read the file from there. Moreover, it's useful when you do not have a persistent memory or you want to work directly with the file.

### Extracting data from PDF

In [48]:
file_to_read = "documents/summaries/resumen_reforma_energetica.pdf"
title = file_to_read.split("/")[-1].split(".")[0]

storage_path = f"gs://{gcp_config.BUCKET_NAME}/{file_to_read}"
#Loads in memory a pdf stored in GCS
pdf_bytes = get_file(file_to_read)

# Create a Document object, it can be constructed from a file or from memory
# pymupdf.Document() method is exactly the same as pymupdf.open()
doc = pymupdf.Document(stream = pdf_bytes)

# Reads the PDF with its metadata and creates a list of dictionaries if chunking, or a string with all the content
md_text = pymupdf4llm.to_markdown(
        doc,
        # page_chunks = True, # Create a list of pages of the Document 
        # extract_words=True, # Adds key words to each page dictionary
        show_progress = False,
    )

md_text

'# Resumen Ejecutivo\n\n\n-----\n\n-----\n\n## I. Introducción\n\nLa Reforma Energética es un paso decidido rumbo a la modernización del sector energético de\nnuestro país, sin privatizar las empresas públicas dedicadas a la producción y al aprovechamiento de los hidrocarburos y de la electricidad. La Reforma Energética, tanto constitucional como a\nnivel legistlación secundarias, surge del estudio y valoración de las distintas iniciativas presentadas por los partidos políticos representados en el Congreso.\n\n### La Reforma Energética tiene los siguientes objetivos y premisas fundamentales:\n\n1. Mantener la propiedad de la Nación sobre los hidrocarburos que se encuentran en el subsuelo.\n2. Modernizar y fortalecer, sin privatizar, a Petróleos Mexicanos (Pemex) y a la Comisión Federal de Electricidad (CFE) como Empresas Productivas del Estado, 100% públicas y 100%\nmexicanas.\n3. Reducir la exposición del país a los riesgos financieros, geológicos y ambientales en las actividades de e

### Chunking the md_text

In this case, we will use the text splitters from [langchain](https://python.langchain.com/api_reference/text_splitters/index.html). Mainly,we will be using the [Markdown](https://python.langchain.com/docs/how_to/markdown_header_metadata_splitter/) and the [RecursiveCharacterTextSplitter](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/) ones.

By default, MarkdownHeaderTextSplitter strips headers being split on from the output chunk's content. This can be disabled by setting: *strip_headers = False*, also, it strips white spaces and new lines. To preserve the original formatting of your Markdown documents, checkout [ExperimentalMarkdownSyntaxTextSplitter](https://python.langchain.com/api_reference/text_splitters/markdown/langchain_text_splitters.markdown.ExperimentalMarkdownSyntaxTextSplitter.html)

In [49]:
# Choose the headers to split on
headers_to_split_on=[("#", 'Header 1'), ("##", "Header 2"), ("###", "Header 3"), ("####", "Header 4"), ("#####", "Header 5")]

# Initialize a MarkdownHeaderTextSplitter object
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on, strip_headers=False)

# Split the text
md_header_splits = markdown_splitter.split_text(md_text)

print(len(md_header_splits))
md_header_splits

39


[Document(metadata={'Header 1': 'Resumen Ejecutivo'}, page_content='# Resumen Ejecutivo  \n-----  \n-----'),
 Document(metadata={'Header 1': 'Resumen Ejecutivo', 'Header 2': 'I. Introducción'}, page_content='## I. Introducción  \nLa Reforma Energética es un paso decidido rumbo a la modernización del sector energético de\nnuestro país, sin privatizar las empresas públicas dedicadas a la producción y al aprovechamiento de los hidrocarburos y de la electricidad. La Reforma Energética, tanto constitucional como a\nnivel legistlación secundarias, surge del estudio y valoración de las distintas iniciativas presentadas por los partidos políticos representados en el Congreso.'),
 Document(metadata={'Header 1': 'Resumen Ejecutivo', 'Header 2': 'I. Introducción', 'Header 3': 'La Reforma Energética tiene los siguientes objetivos y premisas fundamentales:'}, page_content='### La Reforma Energética tiene los siguientes objetivos y premisas fundamentales:  \n1. Mantener la propiedad de la Nación sob

Once the data has been splitted into different chunks, we can split them more to adjust it to a specific chunk size and also specify the chunk overlap per each division already created. To do so, we can then apply any text splitter we want, such as RecursiveCharacterTextSplitter

- **Chunking depends of how many tokens an embedding model supports**

There's a [HuggingFace dashboard](https://huggingface.co/spaces/mteb/leaderboard) that compares the performacne of different embedding models. Some metrics to focus on are:

- Number of Parameters: 

    A higher value means the model requires more CPU/GPU memory to run

- Embedding Dimension:

    The dimension of the vectors produced

- Max tokens:

    How many tokens the model can process, the higher the better.


For this time, we'll be using the [*sentence-transformers/all-MiniLM-L6-v2*](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) model:

- Number of Parameters: 22.7M (small)
- Embedding Dimension: 384
- Max tokens: 256

Another model that will be used is [*sentence-transformers/all-mpnet-base-v2*](https://huggingface.co/sentence-transformers/all-mpnet-base-v2):

- Number of Parameters: 109M (medium)
- Embedding Dimension: 768
- Max tokens: 384

In [50]:
chunk_size1 = 256
chunk_overlap1 = 20
separators1 = [r"\n\n"]

text_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size1, chunk_overlap = chunk_overlap1)

chunks_model1 = text_splitter.split_documents(md_header_splits)

len(chunks_model1)

309

In [51]:
chunk_size2 = 384
chunk_overlap2 = 0
separators2 = [r"\n\n"]

text_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size2, chunk_overlap = chunk_overlap2)

chunks_model2 = text_splitter.split_documents(md_header_splits)

len(chunks_model2)

187

In [52]:
print(len(chunks_model1[1].page_content))
print(chunks_model1[1].page_content, "\n\n")

print(len(chunks_model2[1].page_content))
print(chunks_model2[1].page_content)

112
## I. Introducción  
La Reforma Energética es un paso decidido rumbo a la modernización del sector energético de 


304
## I. Introducción  
La Reforma Energética es un paso decidido rumbo a la modernización del sector energético de
nuestro país, sin privatizar las empresas públicas dedicadas a la producción y al aprovechamiento de los hidrocarburos y de la electricidad. La Reforma Energética, tanto constitucional como a


In [53]:
print(len(md_header_splits[1].page_content))
print(md_header_splits[1].page_content)

465
## I. Introducción  
La Reforma Energética es un paso decidido rumbo a la modernización del sector energético de
nuestro país, sin privatizar las empresas públicas dedicadas a la producción y al aprovechamiento de los hidrocarburos y de la electricidad. La Reforma Energética, tanto constitucional como a
nivel legistlación secundarias, surge del estudio y valoración de las distintas iniciativas presentadas por los partidos políticos representados en el Congreso.


In [54]:
# Adding info such as title, and the date of chunking
extra_metadata = {
    "upload_date": datetime.now().strftime(r"%Y-%m%d %H:%M:%S"),
    "title": title,
    "storage_path": storage_path,
    }


text_metadata1 = [doc.metadata for doc in chunks_model1]
text_metadata2 = [doc.metadata for doc in chunks_model2]

In [55]:
# Updating the metadata with the extra metadata. This list will return None, because the update
# method does not return the dictionary itself

for i, chunk_metadata in enumerate(text_metadata1):
    
    # Add the page content in the metadata
    chunk_metadata["data"] = chunks_model1[i].page_content
    
    # Add extra metadata
    chunk_metadata.update(extra_metadata)

for i, chunk_metadata in enumerate(text_metadata2):
    
    # Add the page content in the metadata
    chunk_metadata["data"] = chunks_model2[i].page_content
    
    # Add extra metadata
    chunk_metadata.update(extra_metadata)


In [56]:
text_metadata2[8]

{'Header 1': 'Resumen Ejecutivo',
 'Header 2': 'I. Introducción',
 'Header 3': 'La Reforma Energética tiene los siguientes objetivos y premisas fundamentales:',
 'data': 'la producción de gas natural de los 5 mil 700 millones de pies cúbicos diarios producidos\nactualmente a 8 mil millones en 2018 y a 10 mil 400 millones en 2025.\n4. Generar cerca de un punto porcentual más de crecimiento económico en 2018 y aproximadamente 2 puntos porcentuales más para 2025.',
 'upload_date': '2025-0327 00:04:17',
 'title': 'resumen_reforma_energetica',
 'storage_path': 'gs://rag_llm_energy_expert/documents/summaries/resumen_reforma_energetica.pdf'}

## Embeddings



In [57]:
text_to_embed1 = [{"vector": doc.page_content, "metadata": text_metadata1[i]} for i, doc in enumerate(chunks_model1)]
text_to_embed2 = [{"vector": doc.page_content, "metadata": text_metadata2[i]} for i, doc in enumerate(chunks_model2)]

In [58]:
text_to_embed1[8]

{'vector': '5. Atraer mayor inversión al sector energético mexicano para impulsar el desarrollo del país.\n6. Contar con un mayor abasto de energéticos a mejores precios.\n7. Garantizar estándares internacionales de eficiencia, calidad y confiabilidad de suministro',
 'metadata': {'Header 1': 'Resumen Ejecutivo',
  'Header 2': 'I. Introducción',
  'Header 3': 'La Reforma Energética tiene los siguientes objetivos y premisas fundamentales:',
  'data': '5. Atraer mayor inversión al sector energético mexicano para impulsar el desarrollo del país.\n6. Contar con un mayor abasto de energéticos a mejores precios.\n7. Garantizar estándares internacionales de eficiencia, calidad y confiabilidad de suministro',
  'upload_date': '2025-0327 00:04:17',
  'title': 'resumen_reforma_energetica',
  'storage_path': 'gs://rag_llm_energy_expert/documents/summaries/resumen_reforma_energetica.pdf'}}

In [59]:
text_to_embed2[8]

{'vector': 'la producción de gas natural de los 5 mil 700 millones de pies cúbicos diarios producidos\nactualmente a 8 mil millones en 2018 y a 10 mil 400 millones en 2025.\n4. Generar cerca de un punto porcentual más de crecimiento económico en 2018 y aproximadamente 2 puntos porcentuales más para 2025.',
 'metadata': {'Header 1': 'Resumen Ejecutivo',
  'Header 2': 'I. Introducción',
  'Header 3': 'La Reforma Energética tiene los siguientes objetivos y premisas fundamentales:',
  'data': 'la producción de gas natural de los 5 mil 700 millones de pies cúbicos diarios producidos\nactualmente a 8 mil millones en 2018 y a 10 mil 400 millones en 2025.\n4. Generar cerca de un punto porcentual más de crecimiento económico en 2018 y aproximadamente 2 puntos porcentuales más para 2025.',
  'upload_date': '2025-0327 00:04:17',
  'title': 'resumen_reforma_energetica',
  'storage_path': 'gs://rag_llm_energy_expert/documents/summaries/resumen_reforma_energetica.pdf'}}

In [60]:
model_name1 = 'sentence-transformers/all-MiniLM-L6-v2'
model_name2 = "sentence-transformers/all-mpnet-base-v2"

model1 = SentenceTransformer(model_name1)
model2 = SentenceTransformer(model_name2)

In [61]:
for chunk in text_to_embed1:
    chunk["vector"] = model.encode(chunk["vector"])


for chunk in text_to_embed2:
    chunk["vector"] = model2.encode(chunk["vector"])

In [62]:
print(f"Vector dimension of model 1: {len(text_to_embed1[0]['vector'])}\nVector dimension of model 2: {len(text_to_embed2[0]['vector'])}")

Vector dimension of model 1: 384
Vector dimension of model 2: 768


In [64]:
text_to_embed1[0]

{'vector': array([-8.76180008e-02,  7.05834106e-02,  1.35683194e-02, -7.78957643e-03,
        -3.25322412e-02,  6.38664141e-02,  2.92839911e-02,  7.83082992e-02,
        -2.15472933e-02, -4.08983268e-02,  4.60199900e-02, -1.40162259e-02,
        -1.10061623e-01, -2.49743294e-02, -5.09077013e-02, -5.50034456e-02,
        -4.99976687e-02,  3.19697075e-02,  2.60699894e-02,  4.11323532e-02,
         6.48808479e-02,  2.59540398e-02,  6.19154284e-03,  9.08319897e-04,
        -2.98400559e-02, -1.79804917e-02,  2.11581029e-02, -7.03359954e-03,
         1.13623263e-02, -7.43363351e-02,  3.64518836e-02, -1.02943201e-02,
         4.71359342e-02, -1.36588328e-02,  7.50755658e-03,  6.61125109e-02,
        -5.64336888e-02,  8.60800222e-03,  7.55290361e-03,  2.79205069e-02,
        -8.37179720e-02, -1.47666976e-01, -8.52674395e-02, -6.19034693e-02,
         1.21373180e-02, -7.65783712e-02, -5.16218171e-02,  6.28516376e-02,
        -4.22569253e-02,  1.71450805e-02, -9.01678950e-02, -9.44447070e-02,
  